In [1]:

from DNA import DNA
from Graficador import *
from Seleccion import Seleccion
import pandas as pd
from Cromosoma import Cromosoma

In [2]:
DATOS_EN_TABLA = '../TablaCapitales.xlsx' 
RUTA =  DATOS_EN_TABLA
FILAS_TABLA = 24 
lista_provincias_visitadas = []
cantidad_km = 0 

df = pd.read_excel(DATOS_EN_TABLA)

def devolver_informacion_tabla_crudo():
    df = pd.read_excel(RUTA)
    tabla = df.values 
 
    return tabla

In [3]:
def procesar_informacion_tabla(tabla: pd.DataFrame):
    lista_provincias = list(tabla[:,0])
    
    return [lista_provincias, tabla]

lista_provincias,tabla = procesar_informacion_tabla(devolver_informacion_tabla_crudo())

In [4]:
 

def get_distance(city1: str, city2: str) -> float:
    return tabla[city1][city2]
#
    # return tabla[lista_provincias.index(city1)][lista_provincias.index(city2) +1]


Generando un marco de datos

In [5]:

def generarDataFrame(pob_dec, pob_bin,info_obj,info_fitness):
    data = {'Decimales': pob_dec, 'Binarios': pob_bin, 'FuncionObjetivo':info_obj,"Fitness":info_fitness}
    dataFrame = pd.DataFrame(data)
    #display(dataFrame)
    return dataFrame



In [6]:
def generarEstadisticas(dataFrame):
    indice_cromosoma_max = dataFrame[dataFrame['FuncionObjetivo']==dataFrame['FuncionObjetivo'].max()].index.values[0]
    cromosoma_max = dataFrame['Binarios'][indice_cromosoma_max]
    stats = {'Suma':[ dataFrame['FuncionObjetivo'].sum(), dataFrame['Fitness'].sum() ],
                'Promedio': [ dataFrame['FuncionObjetivo'].mean(), dataFrame['Fitness'].mean() ],
                'Máximo': [ dataFrame['FuncionObjetivo'].max(), dataFrame['Fitness'].max() ], 
                'Mínimo': [ dataFrame['FuncionObjetivo'].min(), dataFrame['Fitness'].min() ]}
    dataFrame_stats = pd.DataFrame(stats, index = ['Función objetivo', 'Fitness'])
 
    return dataFrame_stats, stats, cromosoma_max 

In [7]:
estadisticas = { "CromosomaMáximo":[],"Máximo":[], "Mínimo":[], "Promedio":[]}


In [8]:
def agregar_stats(stats,cromosoma_max):
    estadisticas["CromosomaMáximo"].append(cromosoma_max)
    estadisticas["Máximo"].append(stats['Máximo'][0])
    estadisticas["Mínimo"].append(stats['Mínimo'][0])
    estadisticas["Promedio"].append(stats['Promedio'][0])
    

In [9]:
def almacenar_informacion(DNA1):
    poblacion_info,info_decimal,info_fitness,info_objetivo = DNA1.get_info_poblacion()
    data_frame = generarDataFrame(info_decimal,poblacion_info,info_objetivo,info_fitness)
    data_frame_stats,stats,cromosoma_max = generarEstadisticas(data_frame)
    agregar_stats(stats,cromosoma_max)

In [10]:
def ciclo(poblacion,fObjetivo,fFitness,numero_ciclo):
    DNA1 =  DNA(poblacion)
    elites = []
    res_objetivo = DNA1.calcular_objetivo(fObjetivo)
    res_fitness = DNA1.calcular_fitness(fFitness)

    almacenar_informacion(DNA1)
    # graficar_por_iteracion(res_objetivo,res_fitness)
    elites = Seleccion.elitismo(DNA1.poblacion,2) # NO ES UN METODO DE SELCCION
    cantidad = 10-len(elites)
    
    # cromosomas_padres = Seleccion.torneo(DNA1.poblacion,cantidad)
    cromosomas_padres = Seleccion.ruleta(DNA1.poblacion,vueltas=cantidad)
    cromosomas_hijos = DNA.cruzar_poblacion_numeros(cromosomas_padres)    

    if len(cromosomas_hijos)>0:
        nueva_poblacion = cromosomas_hijos + elites

    # print(len(nueva_poblacion))
    return nueva_poblacion


In [11]:
ciclos = 200
cantidad_cromosomas = 50
funcion_objetivo = lambda x,y: get_distance(x,y)       #  esto tiene que devolver la distancia, no?
fitness = lambda obj,sum_obj: obj/sum_obj   # distancia total?
Cromosoma.setProbMutacion(0.05)
Cromosoma.setProbCrossover(0.75)
poblacion = DNA.generar_poblacion_numeros(cantidad_cromosomas)
# poblaciones = []

# poblaciones.append(poblacion)


for i in range(ciclos):
    nueva_poblacion = ciclo(poblacion,funcion_objetivo,fitness,i)
    poblacion = nueva_poblacion
    # poblaciones.append(poblacion)    
    # print(f'-----------{i}------------')
hacerGrafico(estadisticas, ciclos)
display(hacerTabla(estadisticas, ciclos))


ValueError: 20 is not in list